# notebook to develop efficient calculation of history term
putting Rep = 0 makes mei, basset and Talaei equal :)

In [ ]:
#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2


import numpy as np
import xarray as xr
from datetime import datetime, timedelta
from scipy.interpolate import CubicSpline # maybe not needed
import scipy.special as sc
import matplotlib.pyplot as plt

import sys
sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
from particle_characteristics_functions import stokes_relaxation_time, Re_particle, factor_drag_white1991, diffusion_time, slip_force
from history_term_functions import Basset_kernel, Mei1992_kernel, history_timescale, Daitche, Hinsberg, f_Mei1992, History_Force_Hinsberg_Mei_kernel, History_Force_Hinsberg_Basset_kernel
from analysis_functions_xr import derivative_backward, calc_tidal_av
from analysis_functions import make_PDF, make_lognormal_PDFd

In [ ]:
# settings of data
starttime = datetime(2023,9,1,0,0,0)
runtime = timedelta(days =2)
endtime = starttime + runtime
loc = 'NWES'
land_handling = 'anti_beaching'
coriolis = True
B = 0.68
tau = 2994.76
Replist = [0,10,100,450,1000]
starttimes = [datetime(2023, 9, 1, 0, 0, 0, 0)]
nparticles = 52511
chunck_time = 100
coriolis = True
gradient = True
dt = timedelta(minutes=5).seconds
Tmax = 12 * 48
n_resample = 10 
dt_resample = dt /n_resample

sec_in_min = 60
min_in_hour = 60
sec_in_hour = sec_in_min * min_in_hour


In [ ]:
# import data set names and directories
base_directory = '/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/test_history_term/'
basefile_Rep_constant = (base_directory + '{particle_type}/{loc}_'
                 'start{y_s:04d}_{m_s:02d}_{d_s:02d}_'
                 'end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_'
                 '_Rep_{Rep:04d}_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')

basefile_Rep_drag = (base_directory + '{particle_type}/{loc}_start{y_s:04d}_{m_s:02d}_{d_s:02d}'
                 '_end{y_e:04d}_{m_e:02d}_{d_e:02d}_RK4_B{B:04d}_tau{tau:04d}_{land_handling}_cor_{coriolis}_gradient_{gradient}.zarr')

basefiles={
           'inertial_Rep_constant':basefile_Rep_constant,
           'inertial_SM_Rep_constant':basefile_Rep_constant, 
           'inertial_SM_drag_Rep':basefile_Rep_drag,
           'inertial_drag_Rep':basefile_Rep_drag}

particle_types = ['inertial_drag_Rep','inertial_Rep_constant']#,'inertial_Rep_constant'] # 
names = {'inertial_Rep_constant':'MR',
         'inertial_SM_Rep_constant':'SM MR', 
         'inertial_SM_drag_Rep':'SM MR flexible Re$_p$',
         'inertial_drag_Rep':'MR flexible Re$_p$'}
Rep_dict = {'inertial_Rep_constant':Replist,
         'inertial_SM_Rep_constant':Replist,
         'inertial_SM_drag_Rep':[None],
         'inertial_drag_Rep':[None]}

In [ ]:

# import data
# create data directory
data = {}
for pt in particle_types:
    data[pt]={}


for pt in particle_types:
    for Rep in Rep_dict[pt]:
        file = basefiles[pt].format(loc=loc,
                                   y_s=starttime.year,
                                   m_s=starttime.month,
                                   d_s=starttime.day,
                                   y_e=endtime.year,
                                   m_e=endtime.month,
                                   d_e=endtime.day,
                                   B = int(B * 1000), 
                                   tau = int(tau ),
                                   land_handling = land_handling, 
                                   coriolis = coriolis,
                                   gradient = gradient,
                                   particle_type = pt,
                                   Rep = Rep)
        ds = xr.open_dataset(file,
                             engine='zarr',
                             chunks={'trajectory':nparticles, 'obs':chunck_time},
                             drop_variables=['B','tau','z'],
                             decode_times=False) 

        Uslip = np.sqrt(ds.uslip**2 + ds.vslip**2) 
        ds = ds.assign({'Uslip':Uslip})
        data[pt][Rep]= ds 

        

In [ ]:
def resample_time(ds, n_resample):
    # Convert obs to numpy array
    obs = ds['obs']
    obs_vals = obs.values
    # Create new obs_resampled n_resample x the original points
    obs_resampled = np.linspace(obs_vals.min(), obs_vals.max(), num=(len(obs_vals) - 1) * n_resample + 1)
    time_resampled = ds['time'].interp(obs=('obs_resampled', obs_resampled))
    ds = ds.assign_coords(obs_resampled=obs_resampled)
    ds['time_resampled'] = time_resampled
    return ds 

def cs_resample_and_derivative(v, t, tresample):
    """
    function that takes an xarray dataarray with time coordinates and returns it resampled function and derivative 
    """
    mask = ~np.isnan(v)
    cs = CubicSpline(t[mask],v[mask])
    data_resampled = cs(tresample)
    data_derivative_resampled = cs.derivative()(tresample)
    return data_resampled, data_derivative_resampled

def cs_resample(v, t, tresample):
    """
    function that takes an xarray dataarray with time coordinates and returns it resampled function
    """
    mask = ~np.isnan(v)
    cs = CubicSpline(t[mask],v[mask])
    data_resampled = cs(tresample)
    return data_resampled


omega_earth =  7.2921e-5 #[rad/sec]

def velocity_factor(uslip, vslip, der_uslip, der_vslip, lat, omega_earth):
    "for now without factor 1/2"
    f_rotation = 2 * omega_earth * np.sin(np.pi * lat /180)
    vel_x = der_uslip - f_rotation * vslip
    vel_y = der_vslip + f_rotation * uslip
    return [vel_x, vel_y]


def f_basset(uslip, vslip, der_uslip, der_vslip, lat, omega_earth, nu, d):
    # Rep = np.mean(Rep)
    factor =  (d**2 / (4 * np.pi * nu) )**(1/2)

    vel_vec = velocity_factor(uslip,vslip,der_uslip,der_vslip,lat,omega_earth)
    return factor * vel_vec[0], factor * vel_vec[1]

def f_mei(t, s, uslip, vslip, der_uslip, der_vslip, lat, omega_earth, c1, c2, nu, d):
    Rep = np.sqrt(uslip**2 + vslip**2) * d / nu
    # Rep = 0 # for testing
    # Rep = np.mean(Rep).values
    A = (4 * np.pi * nu / (d*d))**(1/(2*c1))
    fh = (0.75 + c2*Rep)**3
    B = (np.pi * nu * nu * (t - s )**(3/2) /(fh * d**4) * Rep**3)**(1/c1)
    factor = (A + B)**(-c1)
    vel_vec = velocity_factor(uslip,vslip,der_uslip,der_vslip,lat,omega_earth)
    return factor * vel_vec[0], factor * vel_vec[1]

def f_talaei(t, s, uslip, vslip, der_uslip, der_vslip, lat, omega_earth, nu, d):
    """ f(s) so velocity lists are indexed by s"""
    
    Rep = np.sqrt(uslip**2 + vslip**2) * d / nu
    # Rep = 0 # for testing
    cs = factor_drag_white1991(Rep)
    A = (4 * np.pi * nu / (d*d))**(1/2)
    B = np.sqrt(4 * np.pi * (t - s ))**(1/2) /(d**2 ) * Rep
    factor = (A + B)**(-1)
    vel_vec = velocity_factor(uslip,vslip,der_uslip,der_vslip,lat,omega_earth)
    return cs * factor * vel_vec[0],cs * factor * vel_vec[1]


def trapezoidal_coefficients(N,delta_t):
    """
    trapezoidal coefficients based on method by hinsberg et all 2011
    """
    G = []
    G0 = 4/3 * np.sqrt(delta_t)
    G.append(G0)
    for k in range(1,N):
        Gk = np.sqrt(delta_t)*((k+4/3)/((k+1)**(3/2)+(k+3/2) * (np.sqrt(k)))
                                + (k-4/3)/((k-1)**(3/2)+(k-3/2)*np.sqrt(k)))
        G.append(Gk)
    GN = np.sqrt(delta_t) * (N - 4/3) / ((N-1)**(3/2)+(N-3/2)*np.sqrt(N))
    G.append(GN)
    return G 


    





In [ ]:
ds = data['inertial_drag_Rep'][None]

ds = ds.isel(obs = slice(0,Tmax)).load()
n_resample = 10
ds = resample_time(ds, 10)


# ds_select = ds_select.isel(trajectory=slice(0,5)).load()
dt = 300 #s 
dt_resample =dt/ n_resample 
# ds_select =

In [ ]:
da_uslip_cs, da_uslip_cs_derivative = xr.apply_ufunc(cs_resample_and_derivative,
                                                     ds.uslip,dt * ds.obs.values,dt_resample *  ds.obs_resampled.values, 
                                                     input_core_dims =[["obs"],["obs"],["obs_resampled"]],
                                                     output_core_dims=[["obs_resampled"],["obs_resampled"]],
                                                     vectorize=True,join="override")
da_vslip_cs, da_vslip_cs_derivative = xr.apply_ufunc(cs_resample_and_derivative,
                                                     ds.vslip,dt * ds.obs.values,dt_resample *  ds.obs_resampled.values, 
                                                     input_core_dims =[["obs"],["obs"],["obs_resampled"]],
                                                     output_core_dims=[["obs_resampled"],["obs_resampled"]],
                                                     vectorize=True,join="override")
da_lat_resampled = xr.apply_ufunc(cs_resample,
                                                     ds.lat,dt * ds.obs.values,dt_resample *  ds.obs_resampled.values, 
                                                     input_core_dims =[["obs"],["obs"],["obs_resampled"]],
                                                     output_core_dims=[["obs_resampled"]],
                                                     vectorize=True,join="override")
ds = ds.assign({'uslip_resampled':da_uslip_cs,
                'vslip_resampled':da_vslip_cs,
                'der_uslip_resampled':da_uslip_cs_derivative,
                'der_vslip_resampled':da_vslip_cs_derivative,
                'lat_resampled':da_lat_resampled})

In [ ]:
rho_water = 1027 # kg/m3 https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
dynamic_viscosity_water = 1.41 * 10**(-3) # kg/(ms) https://www.engineeringtoolbox.com/sea-water-properties-d_840.html (at 10 deg)
kinematic_viscosity_water = dynamic_viscosity_water / rho_water
diameter =0.25 # m
B=0.68



cs = {'Mei':{'c1':2,'c2':0.105},
             'Kim':{'c1':2.5,'c2':0.126},
             'Dorgan':{'c1':2.5,'c2':0.2}}



In [ ]:
i=500
Nwindow = 500
ds_test = ds.isel(trajectory=1)

omega_earth =  7.2921e-5 #[rad/sec]

time_i = ds_test.time.isel(obs = i).values
ds_test = ds_test.isel(obs_resampled =slice(i*n_resample-Nwindow-1,i*n_resample))
Glist =np.array(trapezoidal_coefficients(Nwindow,dt_resample))
Glist_r = Glist[::-1]

fmei_listx, fmei_listy = f_mei(t = time_i,s = np.arange(0,Nwindow+1,1)*dt_resample,
uslip = ds_test.uslip_resampled, vslip =  ds_test.vslip_resampled,  
      der_uslip=ds_test.der_uslip_resampled,der_vslip=ds_test.der_vslip_resampled,
      lat = ds_test.lat_resampled, omega_earth=omega_earth, 
      nu = kinematic_viscosity_water,
      d = diameter,
      c1 =cs['Dorgan']['c1'], c2 = cs['Dorgan']['c2'])


ftalaei_listx, ftalaei_listy = f_talaei(t = time_i,s = np.arange(0,Nwindow+1,1)*dt_resample,
      uslip = ds_test.uslip_resampled, vslip =  ds_test.vslip_resampled,  
      der_uslip=ds_test.der_uslip_resampled,der_vslip=ds_test.der_vslip_resampled,
      lat = ds_test.lat_resampled, omega_earth=omega_earth, 
      nu = kinematic_viscosity_water,
      d = diameter)


f_basset_listx, f_basset_listy = f_basset(uslip = ds_test.uslip_resampled, vslip =  ds_test.vslip_resampled,  
      der_uslip=ds_test.der_uslip_resampled,der_vslip=ds_test.der_vslip_resampled,
      lat = ds_test.lat_resampled, omega_earth=omega_earth, 
      nu = kinematic_viscosity_water,
      d = diameter)




In [ ]:
Hmei = [np.sum(fmei_listx*Glist_r).values,np.sum(fmei_listy*Glist_r).values]
Htalaei = [np.sum(ftalaei_listx*Glist_r).values,np.sum(ftalaei_listy*Glist_r).values]
Hbasset = [np.sum(f_basset_listx*Glist_r).values,np.sum(f_basset_listy*Glist_r).values]

In [ ]:
print(Hmei)
print(Htalaei)
print(Hbasset)

In [ ]:
# vectorize? 
i=500
Nwindow = 500
Nstart = 2 * int(Nwindow/n_resample) +1
i_array = np.arange(Nstart, ds.obs[-1].values,1) # Nvalid = number of valid obs indices
Glist =np.array(trapezoidal_coefficients(Nwindow,dt_resample))
Glist_r = Glist[::-1]
slist = np.arange(0,Nwindow+1,1)*dt_resample
# ds_test = ds.isel(trajectory=1)

omega_earth =  7.2921e-5 #[rad/sec]
H_mei_x_list = []
H_mei_y_list = [ ]
for i in i_array[0:10]:
    if(i%10==0):
        print(i)
    time_i = ds.time.isel(obs = i)
    ds_time_i = ds.isel(obs_resampled =slice(i*n_resample-Nwindow-1,i*n_resample))


    fmei_x, fmei_y = xr.apply_ufunc(f_mei,time_i, slist, 
                                    ds_time_i.uslip_resampled, ds_time_i.vslip_resampled,
                                    ds_time_i.der_uslip_resampled, ds_time_i.der_vslip_resampled, 
                                    ds_time_i.lat_resampled, omega_earth,
                                    cs['Dorgan']['c1'], cs['Dorgan']['c2'], 
                                    kinematic_viscosity_water,diameter ,
                                    input_core_dims =[[],["obs_resampled"],["obs_resampled"],["obs_resampled"],["obs_resampled"],["obs_resampled"],["obs_resampled"],[],[],[],[],[]],
                                                        output_core_dims=[["obs_resampled"],["obs_resampled"]],
                                                        vectorize=True)#,join="override")

    hmei_x = (Glist_r * fmei_x).sum(dim='obs_resampled').values
    hmei_y = (Glist_r * fmei_y).sum(dim='obs_resampled').values
    H_mei_x_list.append(hmei_x)
    H_mei_y_list.append(hmei_y)


# now do for every obs (maybe even with for loop? )    


In [ ]:
ds

In [ ]:
np.array(H_mei_y_list)[:,0]

In [ ]:
plt.plot(np.array(H_mei_y_list)[:,0],'--o')

In [ ]:
test = fmei_x.isel(trajectory=1)
print(np.sum(test*Glist_r).values)

In [ ]:
plt.plot(test)
plt.plot(f_basset_listx,'--')
# plt.plot(fmei_listx,'--')


In [ ]:

i_array = np.arange(Nstart, ds.obs[-1].values,1) # Nvalid = number of valid obs indices

In [ ]:
i_array

In [ ]:
results_x =[]
results_y = []
for i in i_array: 
    print(i)
    result_x, result_y = compute_fmei_at_i(i=i, ds=ds,
        Glist_r=Glist_r,
        slist=slist,
        omega_earth=omega_earth,
        c1=cs['Dorgan']['c1'],
        c2=cs['Dorgan']['c2'],
        nu=kinematic_viscosity_water,
        diameter=diameter)
    results_x.append(result_x)
    results_y.append(result_y)
    

In [ ]:
pt = 'inertial_Rep_constant'
ds_history = xr.open_dataset(f'/storage/shared/oceanparcels/output_data/data_Meike/MR_advection/NWES/{pt}/history_term.netcdf')

In [ ]:
ds_history

In [ ]:
ds_history.History_Mei_x.isel(trajectory=1).plot()
ds_history.History_Mei_y.isel(trajectory=1).plot()
ds_history.History_Basset_x.isel(trajectory=1).plot()
ds_history.History_Basset_y.isel(trajectory=1).plot()
ds_history.History_Talaei_x.isel(trajectory=1).plot()
ds_history.History_Talaei_y.isel(trajectory=1).plot()